PYTORCH로 딥러닝하기 60분만에 끝장내기

y=sin(x) 을 예측할 수 있도록, -\pi−π 부터 pipi 까지 유클리드 거리(Euclidean distance)를 최소화하도록 3차 다항식을 학습합니다.

이번에는 사용자가 새롭게 정의한 Module의 하위 클래스(subclass)로 모델을 정의합니다. 기존 Module들을 사용하는 간단한 구성보다 더 복잡한 모델을 원한다면, 이 방법으로 모델을 정의하면 됩니

In [29]:
import torch
import math

class Polynomial3(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        return self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3

    def string(self):
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3'



x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

model = Polynomial3()

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)

for t in range(2000):
    y_pred = model(x)

    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step() # 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정합니다


print(f'Result: {model.string()}')

99 74.57759094238281
199 52.30561828613281
299 37.57805252075195
399 27.838134765625
499 21.396883010864258
599 17.13693618774414
699 14.319591522216797
799 12.45635986328125
899 11.224111557006836
999 10.409117698669434
1099 9.870105743408203
1199 9.513607025146484
1299 9.277815818786621
1399 9.121864318847656
1499 9.018718719482422
1599 8.950490951538086
1699 8.905362129211426
1799 8.875509262084961
1899 8.85576057434082
1999 8.84269905090332
Result: y = -0.0004430137050803751 + 0.8518458008766174 x + 7.642935088369995e-05 x^2 + -0.09263411164283752 x^3
